In [2]:
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

tags_url = "https://www.instagram.com/explore/tags/"
my_tag = 'seoul'
url = tags_url + my_tag

DRIVER_DIR = 'Downloads/chromedriver'
driver = webdriver.Chrome(DRIVER_DIR)
driver.implicitly_wait(3)

driver.get('chrome://settings/')
driver.execute_script('chrome.settingsPrivate.setDefaultZoom(0.25);')
driver.get(url)

elem = driver.find_element_by_tag_name("body")
link_list = []
num_of_crawling_pages = 10

pagedowns = 1
while pagedowns < num_of_crawling_pages:
        elem.send_keys(Keys.PAGE_DOWN)
        time.sleep(0.5)
        elem.send_keys(Keys.PAGE_DOWN)
        time.sleep(0.5)
        elem.send_keys(Keys.PAGE_DOWN)
        time.sleep(0.5)
        elem.send_keys(Keys.PAGE_DOWN)
        time.sleep(0.5)
        elem.send_keys(Keys.PAGE_DOWN)
        time.sleep(0.5)
        elem.send_keys(Keys.PAGE_DOWN)
        time.sleep(1)
        links = driver.find_elements_by_css_selector('div.v1Nh3 > a')
        for i in links:
            link_list.append(i.get_attribute('href'))
        pagedowns += 1

# 크롤링한 결과를 확인해 봅니다.
print('중복링크 개수', len(link_list))

set_link_list = list(set(link_list))
print('유니크링크 개수', len(set_link_list))

print('유니크링크 구성')
print(set_link_list[:10])

In [7]:
first_url = set_link_list[0]
print(first_url)

https://www.instagram.com/p/BsS4zx3BxHC/


In [8]:
import requests

req = requests.get(first_url)
html = req.text
header = req.headers
status = req.status_code
is_ok = req.ok

In [9]:
print(html)

<!DOCTYPE html>
<html lang="en" class="no-js not-logged-in client-root">
    <head>
        <meta charset="utf-8">
        <meta http-equiv="X-UA-Compatible" content="IE=edge">

        <title>
💋X.O.X.O.💋 on Instagram: “CNY ❤️💥 紅一番 •••••••••••••••••••••••••••••••• 🧸❜❜xoxo235xoxo 🧳利時商場235號鋪 📲WTSAPP 90192719(PHOEBE) 📲WTSAPP 95504101(CECI)…”
</title>

        
        <meta name="robots" content="noimageindex, noarchive">
        <meta name="mobile-web-app-capable" content="yes">
        <meta name="theme-color" content="#000000">
        <meta id="viewport" name="viewport" content="width=device-width, initial-scale=1, minimum-scale=1, maximum-scale=1, viewport-fit=cover">
        <link rel="manifest" href="/data/manifest.json">

        
          <link href="https://graph.instagram.com" rel="preconnect" crossorigin>
        

        <link rel="preload" href="/static/bundles/metro/PostPageContainer.js/2c89551b8eb3.js" as="script" type="text/javascript" crossorigin="anonymous" />
       

In [10]:
print(header)

{'Content-Type': 'text/html; charset=utf-8', 'Vary': 'Accept-Language, Cookie, Accept-Encoding', 'Content-Language': 'en', 'Content-Encoding': 'gzip', 'Date': 'Sun, 06 Jan 2019 13:49:56 GMT', 'Strict-Transport-Security': 'max-age=3600', 'Cache-Control': 'private, no-cache, no-store, must-revalidate', 'Pragma': 'no-cache', 'Expires': 'Sat, 01 Jan 2000 00:00:00 GMT', 'X-Frame-Options': 'SAMEORIGIN', 'content-security-policy': "report-uri https://www.instagram.com/security/csp_report/; default-src 'self' https://www.instagram.com; img-src https: data: blob:; font-src https: data:; media-src 'self' blob: https://www.instagram.com https://*.cdninstagram.com https://*.fbcdn.net; manifest-src 'self' https://www.instagram.com; script-src 'self' https://instagram.com https://www.instagram.com https://*.www.instagram.com https://*.cdninstagram.com wss://www.instagram.com https://*.facebook.com https://*.fbcdn.net https://*.facebook.net 'unsafe-inline' 'unsafe-eval' blob:; style-src 'self' https:

In [11]:
print(status)

200


In [12]:
print(is_ok)

True


In [13]:
!pip install bs4

  Running setup.py bdist_wheel for bs4 ... done
  Stored in directory: /Users/Sungjae/Library/Caches/pip/wheels/a0/b0/b2/4f80b9456b87abedbc0bf2d52235414c3467d8889be38dd472
Successfully built bs4


In [49]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(html, 'html.parser')

## 처음에 찾아볼 때, 유저 아이디가 있는 a 태그의 FPmhX 클래스로 들어가려 했으나, JS 로 나중에 동기화 되어 찾을 수 없는듯 하다
# user_id = soup.find_all("a", class_ = "FPmhX") 의 user_id 의 len 은 0이다

## 하지만 다른 한 가지 방법으로 로드하기 이전 metadata 에 해당 게시물의 정보들이 들어있는 듯 하다.
## metadata 에는 본문이 짧게만 나오는 듯 하다
## 전체 본문 내용은 script 를 통해 출력되는 듯 하다. -> 하지만 인코딩이 다른 듯 하다
## display url 을 통해 사진의 링크도 출력된다.
## 결국 BS 로 추출해온 script content 에서 데이터를 뽑아내는게 핵심이다

script_content = soup.find_all("script")

print(script_content)

[<script type="text/javascript">
        (function() {
  var docElement = document.documentElement;
  var classRE = new RegExp('(^|\\s)no-js(\\s|$)');
  var className = docElement.className;
  docElement.className = className.replace(classRE, '$1js$2');
})();
</script>, <script type="text/javascript">
(function() {
  
  if ('PerformanceObserver' in window && 'PerformancePaintTiming' in window) {
    window.__bufferedPerformance = [];
    var ob = new PerformanceObserver(function(e) {
      window.__bufferedPerformance.push.apply(window.__bufferedPerformance,e.getEntries());
    });
    ob.observe({entryTypes:['paint']});
  }
  window.__bufferedErrors = [];
  window.onerror = function(message, url, line, column, error) {
    window.__bufferedErrors.push({
      message: message,
      url: url,
      line: line,
      column: column,
      error: error
    });
    return false;
  };
  window.__initialData = {
    pending: true,
    waiting: []
  };
  function notifyLoaded(item, data) {


In [112]:
## 추출하고자 하는 내용들
## username ( 유저 아이디) -> OK
## contents ( 내용 ) -> OK
## tags ( 태그들 ) -> OK
## image_links ( 이미지 링크들 ) -> OK
## like_num ( 좋아요 개수 )
## comment_num ( 댓글 개수 )


import re

## 해당 script 전체 내용에서 username 의 내용만 추출하여 사용한다
username_p = re.compile("username\":\".*?\"")
username = username_p.findall(str(script_content))
print(username)
print()

['username":"xoxo235xoxo"']



In [113]:
## 이번에는 contents 추출하는 과정입니다
contents_p = re.compile("\"caption\":\".*?\"")
contents = contents_p.findall(str(script_content))
print(contents)
print()

## 이 과정에서 태그가 함께 추출되므로, 이를 분리하는 과정이 약간 필요할 듯 합니다
## 단순하게 # 을 기준으로 태그를 추출할 경우, 본문에 # 을 썼을 경우 문제가 생길 수 있습니다. 
## 현재로서는 따로 태그가 저장되지는 않는 것으로 보입니다

['"caption":"CNY \\u2764\\ufe0f\\ud83d\\udca5 \\u7d05\\u4e00\\u756a\\n\\u2022\\u2022\\u2022\\u2022\\u2022\\u2022\\u2022\\u2022\\u2022\\u2022\\u2022\\u2022\\u2022\\u2022\\u2022\\u2022\\u2022\\u2022\\u2022\\u2022\\u2022\\u2022\\u2022\\u2022\\u2022\\u2022\\u2022\\u2022\\u2022\\u2022\\u2022\\u2022\\n\\ud83e\\uddf8\\u275c\\u275cxoxo235xoxo\\n\\ud83e\\uddf3\\u5229\\u6642\\u5546\\u5834235\\u865f\\u92ea\\n\\ud83d\\udcf2WTSAPP 90192719(PHOEBE)\\n\\ud83d\\udcf2WTSAPP 95504101(CECI)\\n\\u2022\\u2022\\u2022\\u2022\\u2022\\u2022\\u2022\\u2022\\u2022\\u2022\\u2022\\u2022\\u2022\\u2022\\u2022\\u2022\\u2022\\u2022\\u2022\\u2022\\u2022\\u2022\\u2022\\u2022\\u2022\\u2022\\u2022\\u2022\\u2022\\u2022\\u2022\\u2022\\n#style #stylish #coat #instagram #ootd  #girly #girls #korea #spring #dress #skirt #shoes #heels #2019 #jewelry #shopping  #\\u5229\\u6642\\u5546\\u5834 #xoxo #\\u5229\\u6642235 #seoul  #korea #outer  #buyonline #earringshop\\n#chill #\\u97d3\\u570b\\u4ee3\\u8cfc #\\u97d3\\u570b\\u9023\\u7dda 

In [114]:
## 이번에는 이미지 링크 추출 과정입니다
image_links_p = re.compile("\"display_resources\":\[.*?\]")
image_links = image_links_p.findall(str(script_content))
print(image_links)
print()

print(len(image_links))
print()

## 여러개의 이미지 링크들에서 개별 이미지 링크로 변환하는 과정입니다.
first_image_link_p = re.compile("\"src\":\".*?\"")
first_image_link = first_image_link_p.findall(str(image_links[0]))
print(first_image_link[0])
print(first_image_link[1])
print(first_image_link[2])
print(len(first_image_link))
print()

second_image_link_p = re.compile("\"src\":\".*?\"")
second_image_link = second_image_link_p.findall(str(image_links[1]))
print(second_image_link[0])
print(second_image_link[1])
print(second_image_link[2])
print(len(second_image_link))
print()

third_image_link_p = re.compile("\"src\":\".*?\"")
third_image_link = second_image_link_p.findall(str(image_links[2]))
print(third_image_link[0])
print(third_image_link[1])
print(third_image_link[2])
print(len(third_image_link))

## 이미지는 2개였지만, 총 3 종류의 링크가 존재했습니다
## 확인해 보니 3개중 1, 2번째 이미지는 같은 것이며 3번째는 다른 것이었습니다
## 추정해 보면 1번째 이미지는 대표 이미지, 2, 3번째 이미지가 포함된 이미지 2종류라고 생각해볼 수 있습니다
## 또한 각 이미지에 대해 resolution 에 의해 3가지의 다른 형태로 저장된 것을 볼 수 있습니다.
## 1번째는 640, 2번째는 750, 3번째는 원본 크기인것으로 보입니다

['"display_resources":[{"src":"https://scontent-hkg3-2.cdninstagram.com/vp/a6359e2ace60cb7ad744c9e2a6e36fb8/5CB50879/t51.2885-15/sh0.08/e35/s640x640/49371826_2043625165672923_8420299447181332203_n.jpg?_nc_ht=scontent-hkg3-2.cdninstagram.com","config_width":640,"config_height":640},{"src":"https://scontent-hkg3-2.cdninstagram.com/vp/fb30eac32c507cb9d4fb83fea13884ff/5CB9A5BD/t51.2885-15/sh0.08/e35/s750x750/49371826_2043625165672923_8420299447181332203_n.jpg?_nc_ht=scontent-hkg3-2.cdninstagram.com","config_width":750,"config_height":750},{"src":"https://scontent-hkg3-2.cdninstagram.com/vp/0f17cf2dfc1ce217ab8173721577ef63/5CCCF4C3/t51.2885-15/e35/49371826_2043625165672923_8420299447181332203_n.jpg?_nc_ht=scontent-hkg3-2.cdninstagram.com","config_width":1080,"config_height":1080}]', '"display_resources":[{"src":"https://scontent-hkg3-2.cdninstagram.com/vp/a6359e2ace60cb7ad744c9e2a6e36fb8/5CB50879/t51.2885-15/sh0.08/e35/s640x640/49371826_2043625165672923_8420299447181332203_n.jpg?_nc_ht=scon

In [119]:
## 이번에는 like_num, comment_num 추출하는 과정입니다
likes_comments_num_p = re.compile("\"description\":\".*?Comments")
likes_comments_num = likes_comments_num_p.findall(str(script_content))
print(likes_comments_num)
print()

['"description":"4 Likes, 0 Comments']



In [ ]:
## 이제 해당 내용을 정리하여 JSON 객체로 저장하면 됩니다.